<a href="https://colab.research.google.com/github/elf222/elf222/blob/main/Hackathons/Hackathon26_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
working_directory = "/content/drive/MyDrive/ForCollab/Hackathon/"

Setting up data

In [6]:
#scaling numerical data

import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv(working_directory+"train_data.csv")

# Define the range limits
price_min_log, price_max_log = 9, 17
area_min_log, area_max_log = 6, 10
room_min_log, room_max_log = 0, 3

# Apply natural logarithm transformation
data['Area'] = np.log(data['Area'])
data['Price'] = np.log(data['Price'])
data['Bedrooms'] = np.log(data['Bedrooms'])
data['Bathrooms'] = np.log(data['Bathrooms'])

# Scale features to [0, 1]
data['Area'] = (data['Area'] - area_min_log) / (area_max_log - area_min_log)
data['Price'] = (data['Price'] - price_min_log) / (price_max_log - price_min_log)
data['Bedrooms'] = (data['Bedrooms'] - room_min_log) / (room_max_log - room_min_log)
data['Bathrooms'] = (data['Bathrooms'] - room_min_log) / (room_max_log - room_min_log)

# Save the transformed data
data.to_csv(working_directory+"train_data_scaled.csv", index=False)
print("Data preprocessing complete. Transformed dataset saved as 'train_data_scaled.csv'.")

Data preprocessing complete. Transformed dataset saved as 'train_data_scaled.csv'.


In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

# Load the dataset
data = pd.read_csv(working_directory+'train_data_scaled.csv')

# Check that the required columns are present
if 'Area' not in data.columns or 'Price' not in data.columns:
    raise ValueError("Dataset must contain 'Area' and 'Price' columns.")

# Define the features and target variable
X = data[['Area']]
y = data['Price']

# Fit a linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict prices using the line of best fit
data['predicted_price'] = model.predict(X)

# Calculate the absolute distance from the line of best fit
data['distance_from_fit'] = np.abs(data['Price'] - data['predicted_price'])

# Determine the 98th percentile of the distance to exclude the top 2%
threshold = data['distance_from_fit'].quantile(0.98)

# Filter out the top 2% of outliers
initial_count = len(data)
cleaned_data = data[data['distance_from_fit'] <= threshold].drop(columns=['predicted_price', 'distance_from_fit'])
final_count = len(cleaned_data)
deleted_count = initial_count - final_count

# Display progress
for _ in tqdm(range(100), desc="Processing Data"):
    pass

# Print the number of entries deleted
print(f"Entries processed: {initial_count}")
print(f"Entries deleted: {deleted_count}")

# Save the cleaned dataset (optional)
cleaned_data.to_csv(working_directory+'train_data_scaled_cleaned.csv', index=False)

# Display the cleaned data
cleaned_data.head()



Processing Data: 100%|██████████| 100/100 [00:00<00:00, 427118.53it/s]

Entries processed: 428
Entries deleted: 9


,ID,Bedrooms,Bathrooms,Area,ZipCode,Price
0,285,0.536479,0.536479,0.561740,92880,0.535964
1,348,0.231049,0.231049,0.318100,92276,0.321399
2,441,0.462098,0.366204,0.348316,93510,0.545957
4,150,0.462098,0.501359,0.575876,92677,0.675064
5,193,0.366204,0.231049,0.326633,94501,0.595396


In [9]:
#scaling images
import os
import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

In [4]:
image_size = 224
image_scaling_to = image_size//2

In [10]:
# Load the dataset
data = pd.read_csv(working_directory+"train_data_scaled_cleaned.csv")

# Define directories
image_dir = working_directory+"images_train"
output_dir = working_directory+"images_composition"
os.makedirs(output_dir, exist_ok=True)

# List of rooms and positions in the composite image
rooms = ["kitchen", "bedroom", "bathroom", "frontal"]
positions = {"kitchen": (0, 0), "bedroom": (0, image_scaling_to), "bathroom": (image_scaling_to, 0), "frontal": (image_scaling_to, image_scaling_to)}

progress_bar = tqdm(total=428, desc="Progress", unit="step")

# Process images for each ID
for property_id in data["ID"]:
    progress_bar.update(1)
    # Create a blank composite image
    composite_image = Image.new('RGB', (image_size, image_size))

    # Load, resize, and place each room image in the composite
    for room in rooms:
        img_path = os.path.join(image_dir, f"{property_id}_{room}.jpg")
        if os.path.exists(img_path):
            img = Image.open(img_path)
            img = img.resize((image_scaling_to, image_scaling_to), Image.LANCZOS)  # Resize to 32x32
            composite_image.paste(img, positions[room])  # Paste at the designated position
        else:
            print(f"Warning: {img_path} not found.")

    # Save the composite image
    composite_image.save(os.path.join(output_dir, f"{property_id}_composition.jpg"))

print("\nImage formatting complete. Composite images saved in 'images_composite' directory.")


Progress: 100%|██████████| 428/428 [07:47<00:00,  1.07s/step]


Image formatting complete. Composite images saved in 'images_composite' directory.


Setting up the model

In [29]:
from tensorflow.keras.callbacks import Callback
import shutil
import os

# Define the Google Drive destination folder
destination_folder = working_directory+"models/"  # Customize this path if needed
os.makedirs(destination_folder, exist_ok=True)

class SaveEveryNEpochs(Callback):
    def __init__(self, save_model_path, save_every=5):
        super(SaveEveryNEpochs, self).__init__()
        self.save_model_path = save_model_path
        self.save_every = save_every

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_every == 0:  # Save every 'save_every' epochs
            model_save_path = os.path.join(self.save_model_path, f'best_model2_epoch_{epoch + 1}.keras')
            self.model.save(model_save_path)
            print(f"Model saved to {model_save_path} after epoch {epoch + 1}")

save_every_n_epochs = SaveEveryNEpochs(destination_folder, save_every=10)

In [30]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import shutil

In [ ]:
image_size = image_size

In [ ]:
# Load structured data
data = pd.read_csv(working_directory+"train_data_scaled_cleaned.csv")

# Separate features and target variable
X_structured = data[['Bedrooms', 'Bathrooms', 'Area', 'ZipCode']].values
y = data['Price'].values

# Standardize the structured features
scaler = StandardScaler()
X_structured = scaler.fit_transform(X_structured)

# Load images corresponding to each ID
image_data = []
image_ids = data['ID'].tolist()
for img_id in image_ids:
    img_path = f"{working_directory}images_composition/{img_id}_composition.jpg"  # adjust extension if needed
    img = load_img(img_path, target_size=(image_size, image_size))
    img_array = img_to_array(img)
    image_data.append(img_array)
X_images = np.array(image_data) / 255.0  # Normalize pixel values

# Split data into training and testing sets
X_train_img, X_test_img, X_train_struct, X_test_struct, y_train, y_test = train_test_split(
    X_images, X_structured, y, test_size=0.25, random_state=42
)

# Load the VGG16 model without the top layers and freeze its layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
for layer in vgg_base.layers:
    layer.trainable = False

# Define input layers
image_input = Input(shape=(image_size, image_size, 3))
structured_input = Input(shape=(X_structured.shape[1],))

# VGG16-based CNN model for image processing
x = vgg_base(image_input)
x = Flatten()(x)

# Fully connected layers for structured data
y = Dense(64, activation='relu')(structured_input)
y = Dense(32, activation='relu')(y)

# Concatenate the outputs of CNN and structured data layers
combined = Concatenate()([x, y])
z = Dense(128, activation='relu')(combined)
z = Dense(64, activation='relu')(z)
output = Dense(1)(z)

# Create the final model
model = Model(inputs=[image_input, structured_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.005), loss='mse', metrics=['mae'])

# Display training progress
checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss")
early_stopping = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=5e-6)

history = model.fit(
    [X_train_img, X_train_struct],
    y_train,
    validation_data=([X_test_img, X_test_struct], y_test),
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint, early_stopping, save_every_n_epochs, reduce_lr_on_plateau],
    verbose=1
)

Setting up test data:

In [11]:
#scaling numerical data

import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv(working_directory+"test_data_no_target.csv")

# Define the range limits
price_min_log, price_max_log = 9, 17
area_min_log, area_max_log = 6, 10
room_min_log, room_max_log = 0, 3

# Apply natural logarithm transformation
data['Area'] = np.log(data['Area'])
data['Bedrooms'] = np.log(data['Bedrooms'])
data['Bathrooms'] = np.log(data['Bathrooms'])

# Scale features to [0, 1]
data['Area'] = (data['Area'] - area_min_log) / (area_max_log - area_min_log)
data['Bedrooms'] = (data['Bedrooms'] - room_min_log) / (room_max_log - room_min_log)
data['Bathrooms'] = (data['Bathrooms'] - room_min_log) / (room_max_log - room_min_log)

# Save the transformed data
data.to_csv(working_directory+"test_data_scaled.csv", index=False)
print("Data preprocessing complete. Transformed dataset saved as 'test_data_scaled.csv'.")

Data preprocessing complete. Transformed dataset saved as 'test_data_scaled.csv'.


In [15]:
#scaling images
import os
import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

In [13]:
image_size = 224
image_scaling_to = image_size//2

In [16]:
# Load the dataset
data = pd.read_csv(working_directory+"test_data_scaled.csv")

# Define directories
image_dir = working_directory+"images_test"
output_dir = working_directory+"images_composition_test"
os.makedirs(output_dir, exist_ok=True)

# List of rooms and positions in the composite image
rooms = ["kitchen", "bedroom", "bathroom", "frontal"]
positions = {"kitchen": (0, 0), "bedroom": (0, image_scaling_to), "bathroom": (image_scaling_to, 0), "frontal": (image_scaling_to, image_scaling_to)}

progress_bar = tqdm(total=107, desc="Progress", unit="step")

# Process images for each ID
for property_id in data["ID"]:
    progress_bar.update(1)
    # Create a blank composite image
    composite_image = Image.new('RGB', (image_size, image_size))

    # Load, resize, and place each room image in the composite
    for room in rooms:
        img_path = os.path.join(image_dir, f"{property_id}_{room}.jpg")
        if os.path.exists(img_path):
            img = Image.open(img_path)
            img = img.resize((image_scaling_to, image_scaling_to), Image.LANCZOS)  # Resize to 32x32
            composite_image.paste(img, positions[room])  # Paste at the designated position
        else:
            print(f"Warning: {img_path} not found.")

    # Save the composite image
    composite_image.save(os.path.join(output_dir, f"{property_id}_composition.jpg"))

print("\nImage formatting complete. Composite images saved in 'images_composite' directory.")


Progress: 100%|██████████| 107/107 [01:53<00:00,  1.02s/step]


Image formatting complete. Composite images saved in 'images_composite' directory.


Predicting:

In [22]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import os

In [23]:
# Load the trained model
model = load_model("/content/WithGPUForNowTheBest_loss-0.007,mae-0.0672.keras")

# Load test data
test_data = pd.read_csv(working_directory+"test_data_scaled.csv")

# Separate the structured data (without the Price column)
X_test_structured = test_data[['Bedrooms', 'Bathrooms', 'Area', 'ZipCode']].values

# Standardize the structured features using the same scaler used in training
scaler = StandardScaler()
X_test_structured = scaler.fit_transform(X_test_structured)

# Load and preprocess images
image_data = []
image_ids = test_data['ID'].tolist()
for img_id in tqdm(image_ids, desc="Processing Images"):
    img_path = f"{working_directory}images_composition_test/{img_id}_composition.jpg"  # Adjust the extension if needed
    img = load_img(img_path)
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    image_data.append(img_array)

X_test_images = np.array(image_data)

# Run the model to get predictions
predictions = model.predict([X_test_images, X_test_structured])

# Save predictions to "predictions.csv"
prediction_results = pd.DataFrame({'ID': image_ids, 'Price': predictions.flatten()})
prediction_results.to_csv(working_directory+"predictions.csv", index=False)

print("Predictions saved to 'predictions.csv'")



Processing Images: 100%|██████████| 107/107 [00:00<00:00, 167.20it/s]


4/4 ━━━━━━━━━━━━━━━━━━━━ 76s 17s/step
Predictions saved to 'predictions.csv'


In [26]:
#Converting prediction in USD:
import pandas as pd
import numpy as np

# Load the predictions data
predictions = pd.read_csv(working_directory+"predictions.csv")

# Apply the transformation to the 'Price' column
predictions['Price'] = np.exp(predictions['Price'] + 9)

# Load the submission template
submission = pd.read_csv(working_directory+"submission_template.csv")

# Update the 'Price' in submission with the calculated values from predictions based on 'ID'
submission = submission.merge(predictions[['ID', 'Price']], on='ID', how='left')

# Save the updated submission template
submission.to_csv(working_directory+"submission_template.csv", index=False)

In [27]:
import pandas as pd

# Load the CSV file
file_path = working_directory+"submission_template.csv"
df = pd.read_csv(file_path)

# Keep only the "ID" and "Price" columns
df = df[['ID', 'Price']]

# Save the modified DataFrame back to the CSV file
df.to_csv(file_path, index=False)
